In [22]:
import pandas as pd
import numpy as np
from itertools import combinations
from sklearn.cross_validation import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline

In [9]:
product_metadata = pd.read_csv('products.csv')
print(product_metadata.shape)
product_metadata.head()

(49688, 4)


,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


In [8]:
order_prods = pd.read_csv('prior_orders_extended.csv')
print(order_prods.shape)
order_prods.head()

(32434489, 6)


,order_id,product_id,add_to_cart_order,reordered,user_id,order_number
0,2539329,196,1,0,1,1
1,2539329,14084,2,0,1,1
2,2539329,12427,3,0,1,1
3,2539329,26088,4,0,1,1
4,2539329,26405,5,0,1,1


In [10]:
orders = pd.read_csv('orders.csv')
print(orders.shape)
orders.head()

(3421083, 7)


,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


In [117]:
train_orders,valid_orders = train_test_split(orders,train_size=0.9)

In [118]:
train_order_prods = pd.merge(order_prods,train_orders[['order_id']],on='order_id',how='inner')
train_order_prods.shape

(29188852, 6)

In [18]:
train_order_prods.head()

,order_id,product_id,add_to_cart_order,reordered,user_id,order_number
0,3108588,12427,1,1,1,8
1,3108588,196,2,1,1,8
2,3108588,10258,3,1,1,8
3,3108588,25133,4,1,1,8
4,3108588,46149,5,0,1,8


In [119]:
prods_total = train_order_prods.groupby('product_id')['order_id'].count().reset_index()
prods_total.columns = ['product_id','prod_orders']
print(prods_total.shape)
prods_total.head()

(49661, 2)


,product_id,prod_orders
0,1,1646
1,2,83
2,3,251
3,4,296
4,5,13


In [121]:
print(prods_total.shape)
print(prods_total[prods_total['prod_orders']>50].shape)

(49661, 2)
(25418, 2)


In [122]:
filter_order_prods = pd.merge(train_order_prods
                              ,prods_total[prods_total['prod_orders']>10][['product_id']]
                              ,on='product_id'
                              ,how='inner')
filter_order_prods.shape

(29135525, 6)

In [123]:
order_dict = {}
for i,row in filter_order_prods.iterrows():
    if row['order_id'] in order_dict.keys():
        order_dict[row['order_id']].append(row['product_id'])
    else:
        order_dict[row['order_id']] = [row['product_id']]

In [124]:
print(len(order_dict.keys()))
order_dict[3108588]

2893196


[196, 12427, 10258, 25133, 46149, 49235]

In [125]:
copurchase_dict = {}
for order in order_dict.keys():
    if len(order_dict[order])>1:
        for a,b in combinations(order_dict[order],2):
            if (a,b) in copurchase_dict.keys():
                copurchase_dict[(a,b)]+=1
            else:
                copurchase_dict[(a,b)]=1

In [127]:
copurchase_dict[(196,12427)]

874

In [128]:
len(copurchase_dict.keys())

37796289

In [131]:
copurchase_df_dict = {"prod_a":[],"prod_b":[],"copurchase_ct":[]}
for key,value in copurchase_dict.items():
    # a:b
    copurchase_df_dict["prod_a"].append(key[0])
    copurchase_df_dict["prod_b"].append(key[1])
    copurchase_df_dict["copurchase_ct"].append(value)
    # b:a
    copurchase_df_dict["prod_b"].append(key[0])
    copurchase_df_dict["prod_a"].append(key[1])
    copurchase_df_dict["copurchase_ct"].append(value)    

In [132]:
copurchase_df = pd.DataFrame(copurchase_df_dict)
copurchase_df.head()

,copurchase_ct,prod_a,prod_b
0,101,196,14084
1,101,14084,196
2,874,196,12427
3,874,12427,196
4,125,196,26088


In [133]:
copurchase_metrics0 = pd.merge(copurchase_df,prods_total,left_on='prod_a',right_on='product_id')
copurchase_metrics0 = pd.merge(copurchase_metrics0,product_metadata[['product_id','product_name']],on='product_id')
copurchase_metrics0 = copurchase_metrics0[['prod_a','product_name','prod_b','copurchase_ct','prod_orders']]
copurchase_metrics0.columns = ['prod_a','prod_a_name','prod_b','copurchase_ct','prod_a_orders']
copurchase_metrics0.head()

,prod_a,prod_a_name,prod_b,copurchase_ct,prod_a_orders
0,196,Soda,14084,101,32204
1,196,Soda,12427,874,32204
2,196,Soda,26088,125,32204
3,196,Soda,26405,71,32204
4,196,Soda,10258,300,32204


In [134]:
copurchase_metrics1 = pd.merge(copurchase_metrics0
                               ,prods_total,left_on='prod_b',right_on='product_id')
copurchase_metrics1 = pd.merge(copurchase_metrics1
           ,product_metadata[['product_id','product_name']],on='product_id')
copurchase_metrics1 = copurchase_metrics1[['prod_a','prod_a_name','prod_b'
                            ,'product_name','copurchase_ct','prod_a_orders','prod_orders']]
copurchase_metrics1.columns = ['prod_a','prod_a_name','prod_b'
                            ,'prod_b_name','copurchase_ct','prod_a_orders','prod_b_orders']
copurchase_metrics1.head()

,prod_a,prod_a_name,prod_b,prod_b_name,copurchase_ct,prod_a_orders,prod_b_orders
0,196,Soda,14084,Organic Unsweetened Vanilla Almond Milk,101,32204,14308
1,12427,Original Beef Jerky,14084,Organic Unsweetened Vanilla Almond Milk,26,5841,14308
2,26088,Aged White Cheddar Popcorn,14084,Organic Unsweetened Vanilla Almond Milk,21,2267,14308
3,26405,XL Pick-A-Size Paper Towel Rolls,14084,Organic Unsweetened Vanilla Almond Milk,4,1092,14308
4,10258,Pistachios,14084,Organic Unsweetened Vanilla Almond Milk,5,1750,14308


In [143]:
copurchase_metrics1[copurchase_metrics1['copurchase_ct']>10].shape

(5679124, 8)

In [144]:
copurch=copurchase_metrics1[copurchase_metrics1['copurchase_ct']>10]
copurch.to_csv('copurchase_count.csv')